In [31]:
def evaluation_PROTEIN(data_path, dataset, output_path, run):
    run_standard = ['fold1.txt', 'fold2.txt', 'fold3.txt','fold4.txt', 'fold5.txt', 'fold6.txt', 'fold7.txt', 'fold8.txt', 'fold9.txt','fold10.txt']
    data_path_standard = f'Datasets/PROTEIN_DATA/{dataset}/PROTEIN_all/standard/'
    for f in range(10):
        fold = f+1
        
        
        
        df_predicted_processed = pd.read_csv(os.path.join(data_path, run[f]), sep = ',', on_bad_lines= 'skip')
        
        
        df_predicted_processed = df_predicted_processed.dropna()
        df_predicted_processed = df_predicted_processed.drop(df_predicted_processed[df_predicted_processed['Sentence ID'] == 'Done'].index)
        df_predicted_processed.columns = ['Sentence ID','PPI']
        interaction_processed = pd.read_csv(os.path.join(data_path_standard, run_standard[f]), sep = '\t')
        #interaction_processed = interaction_processed [
        y = interaction_processed['isValid']
        #y = convert_bool_to_string(y)
        
        y_pred = df_predicted_processed ['PPI'].astype(bool)
        #y_pred = df_predicted_processed['PPI'].apply(lambda x: True if x == 'TRUE' else False)
        #print(y)
        #print(y_pred)
        
        # Calculate performance metrics
        #precision, recall, f1 = calculate_metrics(y, y_pred)
        accuracy = accuracy_score(y, y_pred)
        precision = precision_score(y, y_pred)
        recall = recall_score(y, y_pred)
        f1 = f1_score(y, y_pred)
        
        print("Fold = ",fold)
        # Print performance metrics
        #print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 Score: {f1}")
        
        print(data_path)
        
        with open(output_path, "a") as f:
          print (data_path, fold,',', "{:.4f}".format(precision), ',', "{:.4f}".format(recall), ',',"{:.4f}".format(f1), file=f)

In [32]:
import os
section_no = "OneSentence_PROTEIN_Prompts"
def output_path(Run_no, dataset, temperature, prompt_no, model_engine):
    base = "Output/" + model_engine + "_"+section_no+"_" + str(temperature) + "/" + str(prompt_no) +"/"+dataset + "/"
    extension_path = dataset + "_T" + str(temperature) + "_" + prompt_no + "_Run" + str(Run_no)+'/'

    # Join the base path and extension path
    Implementation_base_path_output = os.path.join(base, extension_path)
    os.makedirs(Implementation_base_path_output, exist_ok=True)
    #print(Implementation_base_path_output)
    return Implementation_base_path_output

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Define the parameters
temperatures = [0.0]  # Add more temperatures if needed
model_engines = ["gpt-4-0613", "gpt-3.5-turbo-0613"]
datasets = ["LLL", "IEPA", "HPRD50"]
prompt_no = 15

# Iterate over each combination of model engine, dataset, and temperature
for model_engine in model_engines:
    for dataset in datasets:
        for temperature in temperatures:
            for R in range(1, 11):
                query_no = f"Prompt{prompt_no}_OneSentence"
                # Define the runs
                runs = [f'{R}_fold{i}.txt' for i in range(1, 11)]
                # Construct the output path
                output_path_R = f'Output/{model_engine}_OneSentence_PROTEIN_Prompts_{temperature}/{query_no}/{dataset}/Evaluation_{dataset}_T{temperature}_{query_no}_Run{R}.csv'
                Implementation_base_path_output_Run = output_path(R, dataset, temperature, query_no, model_engine)
                evaluation_PROTEIN(Implementation_base_path_output_Run, dataset, output_path_R, runs)


In [ ]:
import pandas as pd
import os

# Define the parameters
model_engines = ["gpt-4-0613", "gpt-3.5-turbo-0613"]
datasets = ["LLL", "IEPA", "HPRD50"]

for model_engine in model_engines:
    for dataset in datasets:
        for prompt_no in range(15, 16):
            # Define the folder path containing the CSV files
            folder_path = f"Output/{model_engine}_OneSentence_PROTEIN_Prompts_0.0/Prompt{prompt_no}_OneSentence/{dataset}/"

            # Create an empty dataframe to store the results
            results_df = pd.DataFrame(columns=["filename", "average precision", "average recall", "average f1 score"])

            # Loop through the CSV files in the folder
            for filename in os.listdir(folder_path):
                if filename.endswith(".csv"):
                    # Read the CSV file into a dataframe
                    df = pd.read_csv(os.path.join(folder_path, filename), header=None, names=["filename", "precision", "recall", "f1 score"])

                    # Calculate the average precision, recall, and f1 score for each file
                    avg_precision = df["precision"].mean()
                    avg_recall = df["recall"].mean()
                    avg_f1_score = df["f1 score"].mean()

                    # Add the results to the results dataframe
                    results_df = results_df.append({
                        "filename": filename,
                        "average precision": avg_precision,
                        "average recall": avg_recall,
                        "average f1 score": avg_f1_score
                    }, ignore_index=True)

            # Extract the "T0.0"-like part of the filenames and store them in a new column called "T_value"
            results_df["T_value"] = results_df["filename"].str.extract("T(\d+\.\d+)")

            # Create a new dataframe with only the shortened filename and the average precision, recall, and f1 score columns
            shortened_df = results_df[["T_value", "average precision", "average recall", "average f1 score"]]

            # Sort the shortened dataframe by the "T_value" column
            shortened_df = shortened_df.sort_values(by="T_value")

            # Convert average precision, recall, and F1 score to percentages and round them
            shortened_df[['average precision', 'average recall', 'average f1 score']] *= 100
            shortened_df[['average precision', 'average recall', 'average f1 score']] = shortened_df[['average precision', 'average recall', 'average f1 score']].round(2)

            # Calculate and print the average scores
            avg_precision = round(shortened_df['average precision'].mean(), 2)
            avg_recall = round(shortened_df['average recall'].mean(), 2)
            avg_f1 = round(shortened_df['average f1 score'].mean(), 2)
            print(f"Model: {model_engine}, Dataset: {dataset}, Average Precision: {avg_precision}, Average Recall: {avg_recall}, Average F1 Score: {avg_f1}")

            # Append the average scores to the DataFrame
            new_row = {'T_value': 'Average', 'average precision': avg_precision, 'average recall': avg_recall, 'average f1 score': avg_f1}
            shortened_df = shortened_df.append(new_row, ignore_index=True)

            # Save the DataFrame as a CSV file
            filepath = os.path.join(folder_path, 'Average_Score.csv')
            shortened_df.to_csv(filepath, index=False)
